In [5]:
import os
import pandas as pd
from datetime import datetime
from atproto import Client
import keyring

# Constants
ROOT_HANDLE = "elaval.bsky.social"
PARQUET_FILE = "followers_log.parquet"
MASTER_FILE = "latest_profiles.parquet"

def get_password():
    """Retrieve the password based on the environment."""
    # Check if running in a GitHub Actions environment
    github_password = os.getenv("BLUESKY_PASSWORD")
    if github_password:
        print("Using GitHub Actions secret for password.")
        return github_password

    # Otherwise, use keyring for local development
    print("Using keyring for password.")
    password = keyring.get_password("bluesky", ROOT_HANDLE)
    if not password:
        raise ValueError(
            "Password not found in keyring. Please set it using keyring for local use or as a GitHub secret for CI/CD."
        )
    return password

def fetch_profile(client, handle):
    """Fetch a user profile from the BlueSky API."""
    try:
        profile = client.get_profile(actor=handle)
        return {
            "id": handle,
            "displayName": profile.display_name or "",
            "description": profile.description or "",
            "followers_count": profile.followers_count or 0,
            "created_at": profile.created_at or "",
            "timestamp": datetime.now(),
        }
    except Exception as e:
        print(f"Error fetching profile for {handle}: {e}")
        return None

def fetch_followers(client, handle):
    """Fetch the list of followers for a given user."""
    followers = []
    cursor = None
    try:
        while True:
            response = client.get_followers(actor=handle, limit=50, cursor=cursor)
            followers.extend([f.handle for f in response.followers])
            if not response.cursor:
                break
            cursor = response.cursor
    except Exception as e:
        print(f"Error fetching followers for {handle}: {e}")
    return followers

def update_master_file(profile, master_data):
    """Update the master file with the latest profile data."""
    existing = master_data[master_data["id"] == profile["id"]]
    if not existing.empty:
        master_data.loc[existing.index] = profile
    else:
        master_data = pd.concat([master_data, pd.DataFrame([profile])], ignore_index=True)
    return master_data

def append_log(handle, followers_count, timestamp, log_data):
    """Append a log entry for followers count and timestamp."""
    log_data = pd.concat([
        log_data,
        pd.DataFrame([{"handle": handle, "followers_count": followers_count, "timestamp": timestamp}])
    ], ignore_index=True)
    return log_data

def main():
    # Load data
    log_data = pd.read_parquet(PARQUET_FILE) if os.path.exists(PARQUET_FILE) else pd.DataFrame()
    master_data = pd.read_parquet(MASTER_FILE) if os.path.exists(MASTER_FILE) else pd.DataFrame()

    # Initialize BlueSky client
    client = Client()
    password = get_password()
    client.login(ROOT_HANDLE, password)

    # Fetch root profile and followers
    root_profile = fetch_profile(client, ROOT_HANDLE)
    if not root_profile:
        print("Failed to fetch root profile. Exiting.")
        return

    followers = fetch_followers(client, ROOT_HANDLE)
    if not followers:
        print("Failed to fetch followers. Exiting.")
        return

    # Update root profile
    log_data = append_log(
        handle=ROOT_HANDLE,
        followers_count=root_profile["followers_count"],
        timestamp=root_profile["timestamp"],
        log_data=log_data,
    )
    master_data = update_master_file(root_profile, master_data)

    # Update followers profiles
    for follower in followers:
        print(f"Processing follower: {follower}")
        profile = fetch_profile(client, follower)
        if profile:
            log_data = append_log(
                handle=follower,
                followers_count=profile["followers_count"],
                timestamp=profile["timestamp"],
                log_data=log_data,
            )
            master_data = update_master_file(profile, master_data)

    # Save updated files
    log_data.to_parquet(PARQUET_FILE, index=False)
    master_data.to_parquet(MASTER_FILE, index=False)
    print(f"Log and master files updated successfully. {len(followers)} followers processed.")

if __name__ == "__main__":
    main()


Using keyring for password.


KeyError: 'id'